In [1]:
import pandas as pd

In [2]:
URL = pd.read_csv("sofabed_url.csv", encoding = "utf-8-sig")["url"]

In [3]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome("/Users/seong-yeseul/Downloads/chromedriver", options = options) #Native
#driver = webdriver.Chrome('chromedriver', options=options)  #Chrome

/var/folders/5j/xbj_5dc97zz4h76plmswc5640000gn/T/ipykernel_6447/1582888393.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/seong-yeseul/Downloads/chromedriver", options = options) #Native


In [15]:
#크롤링 함수 정의 (0102 최종!! 진짜최종!!)
def crawler(URL, driver):
    import pandas as pd
    import numpy as np
    import math
    import time
    import warnings
    warnings.filterwarnings("ignore")
    ## (1) 준비
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')        # Head-less 설정
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome("/Users/seong-yeseul/Downloads/chromedriver", options = options) #Native
    #driver = webdriver.Chrome('chromedriver', options=options)  #Chrome
    result = pd.DataFrame()
    for k, url in enumerate(URL):
        print("url_open... {0}번째 아이템 데이터를 수집합니다...".format(k+1))
        driver.get(url)
        driver.maximize_window()    
        ## (2) 상품 정보 수집 
        #상품명                                         
        item_name = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > h1 > div > span').text
        #침대의 경우 침대가 아닌데 해당 카테고리에 있는 상품 거르기 위해
        #if "테이블" in item_name:
        #    continue
        #elif "트레이" in item_name:
        #    continue

        #상품브랜드
        item_brand = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > h1 > p > a').text
        #상품정가
        #bed_price = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > div.production-selling-header__content > div.production-selling-header__price > span.production-selling-header__price__price-wrap > del > span.number').text
        #상품특가
        try:
            item_sale_price = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > div.production-selling-header__content > div.production-selling-header__price > span.production-selling-header__price__price-wrap > span.production-selling-header__price__price > span.number').text
        except:
            item_sale_price = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/dl/div[1]/dd").text
        #상품할인가(첫구매 할인)
        #bed_sale_price2 = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > div.production-selling-header__content > div.production-selling-header__price > span.production-selling-header__price__coupon > span.number').text
        #상품대분류
        item_category1 = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > nav > ol > li:nth-child(2) > a').text
        #상품중분류
        item_category2 = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > nav > ol > li:nth-child(3) > a').text
        #상품소분류
        try:
            item_category3 = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > nav > ol > li:nth-child(4) > a').text
        except:
            item_category3 = item_category2
        #평균평점
        item_score = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.production-selling-overview.container > div > div.production-selling-overview__content.col-12.col-md-6.col-lg-5 > div.production-selling-header > div.production-selling-header__content > p > a > span.production-selling-header__review__icon')
        item_score = item_score.get_attribute('aria-label')
        #리뷰건수
        item_count = driver.find_element_by_css_selector('body > div.page > div > div.production-selling > div.sticky-container.production-selling-navigation-wrap > div > nav > ol > li:nth-child(2) > a > span').text
        #콤마제거
        if ',' in item_count:
            item_count = item_count.split(',')
            item_count = int(item_count[0]+item_count[1])
        else:
            item_count = int(item_count) 
    
        ## (3) 리뷰 정보 수집
        #리뷰 클릭
        time.sleep(2)
        try:
            element = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div[1]/div[1]/p/a")
        except:
            element = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[4]/div/nav/ol/li[2]/a')
        #리뷰 이동
        time.sleep(1)
        driver.execute_script('arguments[0].click();', element) # 해당 element까지 스크롤 후 클릭 
        
        #리뷰 리스트 정의
        list_user = []
        list_date = []
        list_total =[]
        list_durability = []
        list_price = []
        list_design = []
        list_delivery = []
        list_option = []
        list_review = []
        list_assist = []
    
        #리뷰 크롤링(한 페이지에 5개)
        page_num = math.ceil(item_count/5) #page_num을 그냥 리뷰 개수에 맞게 유동적 조절
        for i in range(1, page_num+1):
            print(i, '번째 페이지입니다.')
            time.sleep(1)
            for j in range(1, 6):
                if (i-1)*5 + j > item_count:
                    break
                try:
                    driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div["+str(j)+"]")
                except:
                    continue
                #닉네임
                time.sleep(1)
                try:
                    user = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[1]/div/p')
                    list_user.append(user[0].text)
                except:
                    print("error raised at {0}th page {1}th item".format(i, j))
                    list_user.append("NULL")
                #작성일자
                try:
                    date = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[1]/div/span')
                    list_date.append(date[0].text)
                except:
                    print("error raised at {0}th page {1}th item".format(i, j))
                    list_date.append("NULL")
                #전체평점
                time.sleep(0.5)
                try:
                    total = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[1]/div/button/span')
                    list_total.append(total[0].get_attribute('aria-label'))
                except:
                    print("error raised at {0}th page {1}th item".format(i, j))
                    list_total.append("NULL")
                #내구성평점
                try:
                    durability = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[2]/div[1]/span[2]')
                    list_durability.append(durability[0].get_attribute('aria-label'))                    
                except:
                    list_durability.append("NULL")
                #가격평점
                time.sleep(0.5)
                try:
                    price = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[2]/div[2]/span[2]')
                    list_price.append(price[0].get_attribute('aria-label'))
                except:
                    list_price.append("NULL")
                #디자인평점
                try:
                    design = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[2]/div[1]/span[4]')
                    list_design.append(design[0].get_attribute('aria-label'))
                except:
                    list_design.append("NULL")
                #배송평점
                time.sleep(0.5)
                try:
                    delivery = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[2]/div[2]/span[4]')
                    list_delivery.append(delivery[0].get_attribute('aria-label'))
                except:
                    list_delivery.append("NULL")
                #구매옵션
                try:
                    option = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[3]')
                    list_option.append(option[0].text)
                except:
                    list_option.append("NULL")
                #리뷰내용
                time.sleep(0.5)
                try:
                    review = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/p')
                    list_review.append(review[0].text)
                except:
                    list_review.append("NULL")
                #도움수 -> 주의! 도움수가 없는 리뷰도 있음
                try:
                    assist = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div['+str(j)+']/article/div[4]/div/span')                    
                    list_assist.append(assist[0].text)
                except:
                    list_assist.append(0)
            
                #화면에 2개씩 보이도록 스크롤 
                if j==2:                                  
                    try:
                        time.sleep(1)
                        element = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div[2]/article')
                        driver.execute_script('arguments[0].scrollIntoView(true);', element)
                    except:
                        time.sleep(1)
                elif j==4:                                  
                    try:
                        time.sleep(1)
                        element = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[3]/div[4]/article')
                        driver.execute_script('arguments[0].scrollIntoView(true);', element)
                    except:
                        time.sleep(1)
        
            #페이지 넘기기                           
            if i != page_num:
                #xpath = "//button[contains(text(),"+str(i+1)+")]"
                #element = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[5]/div/div[1]/div/section[2]/div/ul').find_element_by_xpath(xpath)
                try:
                    xpath = "//section[2]/div/ul/li/button[contains(text(),"+str(i+1)+")]"
                    element = driver.find_element_by_xpath(xpath)
                    time.sleep(0.2)
                    driver.execute_script('arguments[0].click();', element) # 역시 스크롤 후 클릭
                    time.sleep(1)
                except:
                    print("cannot move to {0}th item {1}th page".format(k+1, i+1))
                    continue
        #데이터프레임 저장        
        tmp = pd.DataFrame()
        tmp['user'] = pd.Series(list_user)
        tmp['date'] = pd.Series(list_date)
        tmp['total_score'] = pd.Series(list_total)
        tmp['dur_score'] = pd.Series(list_durability)
        tmp['price_score'] = pd.Series(list_price)
        tmp['design_score'] = pd.Series(list_design)
        tmp['delivery_score'] = pd.Series(list_delivery)
        tmp['options'] = pd.Series(list_option)
        tmp['review'] = pd.Series(list_review)
        tmp['help_count'] = pd.Series(list_assist)
        tmp['item_link'] = url
        tmp['item_name'] = item_name
        tmp["item_brand"] = item_brand
        #tmp["상품 정가"] = item_price
        tmp["item_sale_price"] = item_sale_price
        #tmp["상품 할인가"] = item_sale_price2
        tmp["item_category1"] = item_category1
        tmp["item_category2"] = item_category2
        tmp["item_category3"] = item_category3
        tmp["item_score"] = item_score
        tmp["item_count"] = item_count
    
        ## (4) 최종 크롤링 데이터
        #데이터가 중복 수집될 경우 리뷰 수만큼만 cut
        if item_count < len(tmp):
            tmp = tmp[:item_count]
        print('리뷰 수: {0}, 수집된 리뷰 수: {1}'.format(item_count, len(tmp)))
        #데이터 저장 및 내보내기
        result = pd.concat([result, tmp], axis = 0)
        
    #크롤링 종료
    driver.quit()
    print(result.isnull().sum())
    result = result.dropna(subset = ["user"])
    result = result.reset_index(drop = True)
    print(result.isnull().sum())
    result.to_csv("sofabed_30_50.csv", encoding = "utf-8-sig", index = False)
    
    return result

In [11]:
URL[10:]

10     https://ohou.se/productions/853691/selling?aff...
11     https://ohou.se/productions/16779/selling?affe...
12     https://ohou.se/productions/362149/selling?aff...
13     https://ohou.se/productions/302014/selling?aff...
14     https://ohou.se/productions/490646/selling?aff...
                             ...                        
199    https://ohou.se/productions/504210/selling?aff...
200    https://ohou.se/productions/448521/selling?aff...
201    https://ohou.se/productions/448518/selling?aff...
202    https://ohou.se/productions/368928/selling?aff...
203    https://ohou.se/productions/322228/selling?aff...
Name: url, Length: 194, dtype: object

In [16]:
crawler(URL[30:50], driver)

url_open... 1번째 아이템 데이터를 수집합니다...
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
19 번째 페이지입니다.
20 번째 페이지입니다.
21 번째 페이지입니다.
22 번째 페이지입니다.
23 번째 페이지입니다.
24 번째 페이지입니다.
25 번째 페이지입니다.
26 번째 페이지입니다.
리뷰 수: 130, 수집된 리뷰 수: 130
url_open... 2번째 아이템 데이터를 수집합니다...
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
19 번째 페이지입니다.
20 번째 페이지입니다.
21 번째 페이지입니다.
22 번째 페이지입니다.
23 번째 페이지입니다.
24 번째 페이지입니다.
25 번째 페이지입니다.
리뷰 수: 122, 수집된 리뷰 수: 122
url_open... 3번째 아이템 데이터를 수집합니다...
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페

,user,date,total_score,dur_score,price_score,design_score,delivery_score,options,review,help_count,item_link,item_name,item_brand,item_sale_price,item_category1,item_category2,item_category3,item_score,item_count
0,ᵢₘ_,2021.12.12 ∙ 오늘의집 구매,별점 5.0점,별점 5.0점,별점 5.0점,별점 5.0점,별점 5.0점,그레이,굿굿 앉았을 때 높이가 좌식소파처럼 낮긴 하지만 넓고 튼튼해서 마음에 듭니다! 후기...,0,https://ohou.se/productions/312942/selling?aff...,토마스 방수가죽 3단 접이식 3인용 소파베드,잉글랜더,"179,000",소파,소파베드,소파베드,별점 4.5점,130
1,아나1693,2021.12.04 ∙ 오늘의집 구매,별점 4.5점,별점 5.0점,별점 4.0점,별점 5.0점,별점 4.0점,그레이,너무 좋아요~ 생각보다 배송 빨라요 🤗,0,https://ohou.se/productions/312942/selling?aff...,토마스 방수가죽 3단 접이식 3인용 소파베드,잉글랜더,"179,000",소파,소파베드,소파베드,별점 4.5점,130
2,나는야잰이,2021.08.24 ∙ 오늘의집 구매,별점 3.8점,별점 3.0점,별점 4.0점,별점 5.0점,별점 3.0점,그레이,나름 고민 많이하고 산거라 상세 솔직 리뷰쓸게요 일단 디자인이 깔끔하고 예뻐요 소파...,1,https://ohou.se/productions/312942/selling?aff...,토마스 방수가죽 3단 접이식 3인용 소파베드,잉글랜더,"179,000",소파,소파베드,소파베드,별점 4.5점,130
3,Kjw3678,2021.08.16 ∙ 오늘의집 구매,별점 4.5점,별점 5.0점,별점 5.0점,별점 5.0점,별점 3.0점,그레이,"크고 튼튼해서 괜찮습니다,, 배송이 3주정도 걸리긴하지만 만족합니다",0,https://ohou.se/productions/312942/selling?aff...,토마스 방수가죽 3단 접이식 3인용 소파베드,잉글랜더,"179,000",소파,소파베드,소파베드,별점 4.5점,130
4,Zero0,2021.07.20 ∙ 오늘의집 구매,별점 3.3점,별점 2.0점,별점 3.0점,별점 5.0점,별점 3.0점,그레이,새집에 동생이 많이 놀러와서 침대로도 사용 가능한 소파로 구매했어요 소파로 사용할때...,0,https://ohou.se/productions/312942/selling?aff...,토마스 방수가죽 3단 접이식 3인용 소파베드,잉글랜더,"179,000",소파,소파베드,소파베드,별점 4.5점,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,엔젤 오리,2019.05.15 ∙ 오늘의집 구매,별점 3.8점,별점 5.0점,별점 3.0점,별점 5.0점,별점 2.0점,단품,너무 오래 기다렸네요ㅋ 기다린 만큼 보람은 있었어요ㅋ 크기때문에 고민 진짜 많이 했...,0,https://ohou.se/productions/645679/selling?aff...,알렉스 103 2인 좌식 소파 베드,아르네스튜디오,"200,000",소파,소파베드,소파베드,별점 4.5점,69
1817,츄츄츄!,2019.04.29 ∙ 오늘의집 구매,별점 4.5점,별점 5.0점,별점 5.0점,별점 5.0점,별점 3.0점,단품,2인용사이즈 좋아요 가격대비최고 기대이상으로 만족해요!,0,https://ohou.se/productions/645679/selling?aff...,알렉스 103 2인 좌식 소파 베드,아르네스튜디오,"200,000",소파,소파베드,소파베드,별점 4.5점,69
1818,쏠아1,2019.04.08 ∙ 오늘의집 구매,별점 5.0점,별점 5.0점,별점 5.0점,별점 5.0점,별점 5.0점,단품,벙커침대 1층에 놓으려고 샀는데..너무 편하고 좋아요 덕분에 2층은 아직도 안올라가봤어요,0,https://ohou.se/productions/645679/selling?aff...,알렉스 103 2인 좌식 소파 베드,아르네스튜디오,"200,000",소파,소파베드,소파베드,별점 4.5점,69
1819,꽃피게,2019.02.11 ∙ 오늘의집 구매,별점 3.3점,별점 5.0점,별점 3.0점,별점 3.0점,별점 2.0점,단품,머리부분이 너무 높아서 그부분빼고는 사용하기 좋아요 난방텐트있는 집에는 의자처럼 올...,0,https://ohou.se/productions/645679/selling?aff...,알렉스 103 2인 좌식 소파 베드,아르네스튜디오,"200,000",소파,소파베드,소파베드,별점 4.5점,69
